In [43]:
import pulp

# Supply end: suppliers
supply = {
    "Supplier1": 20,
    "Supplier2": 30
}

# Demand end: consumers
demand = {
    "Consumer1": 25,
    "Consumer2": 15,
    "Consumer3": 10
}

# Cost info (per unit shipping qty)
costs = {
    ("Supplier1", "Consumer1"): 4,
    ("Supplier1", "Consumer2"): 3,
    ("Supplier1", "Consumer3"): 2,
    ("Supplier2", "Consumer1"): 5,
    ("Supplier2", "Consumer2"): 6,
    ("Supplier2", "Consumer3"): 8
}

# Customize a leats-cost for a specific consumer
user_identity = input("Enter your identity (Consumer1, Consumer2, Consumer3): ") # assuming the cost burden is solely on the consumer

# Function to calculate least cost for a specific user
def calculate_least_cost(user_identity):
    user_model = pulp.LpProblem("User Cost Optimization", pulp.LpMinimize)
    user_routes = pulp.LpVariable.dicts("Route", costs, lowBound=0, cat='Continuous')

    if user_identity in supply:
        # Supplier specific model
        user_model += pulp.lpSum([user_routes[(user_identity, d)] * costs[(user_identity, d)] for d in demand]), "User Transportation Cost"
        for s in supply:
            user_model += pulp.lpSum([user_routes[(s, d)] for d in demand]) <= supply[s], f"Supply Constraint {s}"
        for d in demand:
            user_model += pulp.lpSum([user_routes[(s, d)] for s in supply]) >= demand[d], f"Demand Constraint {d}"
    elif user_identity in demand:
        # Consumer specific model
        user_model += pulp.lpSum([user_routes[(s, user_identity)] * costs[(s, user_identity)] for s in supply]), "User Transportation Cost"
        for s in supply:
            user_model += pulp.lpSum([user_routes[(s, d)] for d in demand]) <= supply[s], f"Supply Constraint {s}"
        for d in demand:
            user_model += pulp.lpSum([user_routes[(s, d)] for s in supply]) >= demand[d], f"Demand Constraint {d}"

    user_model.solve()

    if pulp.LpStatus[user_model.status] == "Optimal":
        user_cost = pulp.value(user_model.objective)
        user_solution = { (s, d): user_routes[(s, d)].varValue for (s, d) in costs }
        return user_cost, user_solution
    else:
        return None, None

# Calculate the least cost for the user
user_cost, user_solution = calculate_least_cost(user_identity)

# Print the user's customized solution
if user_cost is not None:
    print(f"\n\033[1mCustomized least-cost shipment plan for \033[94m{user_identity}:\033[0m")
    
    # Calculate the specific cost associated with the user
    if user_identity in supply:
        total_user_cost = sum(user_solution[(user_identity, d)] * costs[(user_identity, d)] for d in demand)
    elif user_identity in demand:
        total_user_cost = sum(user_solution[(s, user_identity)] * costs[(s, user_identity)] for s in supply)
    
    # Calculate total transportation cost
    total_transportation_cost = sum(user_solution[(s, d)] * costs[(s, d)] for (s, d) in costs)
    
    # least-cost shipment plan for the user
    for (s, d) in costs:
        if user_solution[(s, d)] > 0:
            print(f"From {s} to {d}: {user_solution[(s, d)]:.2f} units")
    
    # Costs as per the user-customized model
    print(f"\nThe least transportation cost for \033[94m{user_identity}\033[0m: {total_user_cost:.2f}")
    print(f"Total transportation cost: {total_transportation_cost:.2f}\n")
    
    # Adjusting supply and demand based on the user's optimized solution
    adjusted_supply = supply.copy()
    adjusted_demand = demand.copy()
    for (s, d) in user_solution:
        if user_solution[(s, d)] > 0:
            if s in adjusted_supply:
                adjusted_supply[s] -= user_solution[(s, d)]
            if d in adjusted_demand:
                adjusted_demand[d] -= user_solution[(s, d)]

    # Model for overall optimization
    overall_model = pulp.LpProblem("Overall Cost Optimization", pulp.LpMinimize)
    overall_routes = pulp.LpVariable.dicts("Route", costs, lowBound=0, cat='Continuous')

    # Cost minimization for overall model
    overall_model += pulp.lpSum([overall_routes[(s, d)] * costs[(s, d)] for (s, d) in costs]), "Total Transportation Cost"

    # Supply constraint
    for s in supply:
        overall_model += pulp.lpSum([overall_routes[(s, d)] for d in demand]) <= supply[s], f"Supply Constraint {s}"

    # Demand constraint
    for d in demand:
        overall_model += pulp.lpSum([overall_routes[(s, d)] for s in supply]) >= demand[d], f"Demand Constraint {d}"

    # Solving the overall model
    overall_model.solve()

    # Feasibility testing for overall model
    if pulp.LpStatus[overall_model.status] == "Optimal":
        overall_cost = pulp.value(overall_model.objective)
        overall_solution = { (s, d): overall_routes[(s, d)].varValue for (s, d) in costs }

        # Overall optimized solution
        print(f"\033[92m\033[1m\nStatus: {pulp.LpStatus[overall_model.status]}\033[0m")
        print("\n\033[1mOverall optimized shipment plan:\033[0m")
        for (s, d) in costs:
            print(f"From {s} to {d}: {overall_routes[(s, d)].varValue:.2f} units")

        # Comparing costs for user-customized model and the overall optimized model
        consumer1_cost = sum(overall_solution[(s, d)] * costs[(s, d)] for (s, d) in costs if d == user_identity)
        print(f"\nTransportation cost for \033[94m{user_identity}\033[0m: {consumer1_cost:.2f} (additional cost: {consumer1_cost - total_user_cost:.2f})")
        print(f"The lowest total transportation cost: {overall_cost:.2f} (cost savings: {total_transportation_cost - overall_cost:.2f})")

    else:
        print("No feasible solution found for the overall model")

else:
    print(f"No feasible solution found for {user_identity}")

Enter your identity (Consumer1, Consumer2, Consumer3):  Consumer1



Customized least-cost shipment plan for Consumer1:
From Supplier1 to Consumer1: 20.00 units
From Supplier2 to Consumer1: 5.00 units
From Supplier2 to Consumer2: 15.00 units
From Supplier2 to Consumer3: 10.00 units

The least transportation cost for Consumer1: 105.00
Total transportation cost: 275.00


Status: Optimal

Overall optimized shipment plan:
From Supplier1 to Consumer1: 0.00 units
From Supplier1 to Consumer2: 10.00 units
From Supplier1 to Consumer3: 10.00 units
From Supplier2 to Consumer1: 25.00 units
From Supplier2 to Consumer2: 5.00 units
From Supplier2 to Consumer3: 0.00 units

Transportation cost for Consumer1: 125.00 (additional cost: 20.00)
The lowest total transportation cost: 205.00 (cost savings: 70.00)
